In [1]:
import pandas as pd
import numpy as np
import os
import multiprocessing as mp

from skimage import io
from tqdm.notebook import tqdm

In [2]:
parquet_path = 'dataset/parquet/'
image_path = 'dataset/images/'

file_list = sorted(os.listdir(parquet_path))
file_list = tqdm(file_list)

In [3]:
for file in file_list:
    data = pd.read_parquet(os.path.join(parquet_path, file), engine='pyarrow')
    image_ids = list(data.image_id.values)
    images = data.iloc[:,1:].values.reshape((-1,137,236))
    
    def save_image(x):
        name, img = x
        io.imsave(os.path.join(image_path, f'{name}.png'), img)

    iterator = zip(image_ids, images)

    with mp.Pool(8) as pool:
        iterator = pool.imap_unordered(save_image, iterator)
        iterator = tqdm(iterator, total=len(image_ids), leave=False)
        list(iterator)

ArrowIOError: Invalid parquet file. Corrupt footer.